In [1]:
import os
import cv2
import copy
import glob
import tkinter
import traceback
import matplotlib
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [3]:
# Given an image, find the maximum rectangle in each
def find_max_rectangular_mask(image: np.ndarray) -> np.ndarray:
    rows = len(image)
    if rows == 0:
        return None

    cols = len(image[0])
    max_area = 0
    max_rect = None
    heights = [0] * (cols + 1)
    stack = []

    for row in range(rows):
        for col in range(cols):
            if image[row][col]:
                heights[col] += 1
            else:
                heights[col] = 0

        stack.clear()
        stack.append(-1)

        for col in range(cols + 1):
            while heights[col] < heights[stack[-1]]:
                h = heights[stack.pop()]
                w = col - stack[-1] - 1
                area = h * w
                if area > max_area:
                    max_area = area
                    max_rect = (row - h + 1, stack[-1] + 1, row, col - 1)

            stack.append(col)

    if max_rect is not None:
        mask = np.zeros_like(image, dtype=bool)
        r1, c1, r2, c2 = max_rect
        mask[r1:r2+1, c1:c2+1] = True
        return mask, (r1,r2+1,c1,c2+1)
    else:
        return None
    
# Now we need to do this for every image in the scene 
# Iterate over each channel and apply the transformation
def apply_homography_to_cube(homography: np.ndarray, rgb_img: np.ndarray, hyper_cube: np.ndarray) -> np.ndarray:
    # Create the empty data structure - spatial size of rgb image
    # BUT with the same number of channels as the hypercube contains
    data = np.zeros((rgb_img.shape[0], rgb_img.shape[1], hyper_cube.shape[2]), dtype=np.float16)
    # Apply the warp
    for idx in range(hyper_cube.shape[2]):
        data[:,:,idx] = cv2.warpPerspective(
            hyper_cube[:,:,idx],
            homography,
            (rgb_img.shape[1], rgb_img.shape[0]),
            rgb_img,
            borderMode=cv2.BORDER_TRANSPARENT
        )
    return data

def generate_registered_cube(h_ximea_2_img: np.ndarray, h_imec_2_img: np.ndarray, 
                            im_dest: np.ndarray, ximea_cube: np.ndarray, imec_cube: np.ndarray,
                            mask: np.ndarray=[]) -> np.ndarray:
    # Register the image using the precomputed homographies
    ximea_registered = apply_homography_to_cube(h_ximea_2_img, im_dest, ximea_cube)
    imec_registered = apply_homography_to_cube(h_imec_2_img, im_dest, imec_cube)
    combined_spectra= np.dstack((ximea_registered,imec_registered))
    return combined_spectra

In [19]:
# Create the mask image
ximea_cube = np.load(glob.glob('./VNIR_RAW/*')[0]).reshape((215, 407, 24))
imec_cube =  np.load(glob.glob('./SWIR_RAW/*')[0]).reshape((168, 211, 9))
h_ximea_2_img = np.loadtxt('./calibration/ximea_to_rgb_homography.txt')
h_imec_2_img = np.loadtxt('./calibration/imec_to_rgb_homography.txt')
im_dest = cv2.cvtColor(cv2.imread(glob.glob('./RGB_RAW/*')[0]), cv2.COLOR_BGR2GRAY)
print(im_dest.shape)
ximea_mask = apply_homography_to_cube(h_ximea_2_img, im_dest, np.ones(ximea_cube.shape))[:,:,0].astype(bool)
imec_mask = apply_homography_to_cube(h_imec_2_img, im_dest, np.ones(imec_cube.shape))[:,:,0].astype(bool)
mask = np.bitwise_and(ximea_mask, imec_mask)
rect_mask, bounds = find_max_rectangular_mask(mask)
cv2.imwrite('./calibration/mask_rect.png', rect_mask.astype(int))

(2054, 2464)


True

In [ ]:
# This is the main loop which will generate the registered data
# Generate all the registered hyperspectral datacubes
# Load homography for ximea -> rgb
ximea_to_rgb = np.loadtxt('./calibration/ximea_to_rgb_homography.txt')
# Load homography for imec -> rgb
imec_to_rgb = np.loadtxt('./calibration/imec_to_rgb_homography.txt')
# Image Mask
mask = cv2.imread('./calibration/mask_rect.png', cv2.IMREAD_GRAYSCALE).astype(bool)
pbar = tqdm(glob.glob('./RGB_RAW/*'))
# Create the registered directories
try:
    os.mkdir(os.path.join('SEGMENTATIONS_REGISTERED'))
    os.mkdir(os.path.join('RGB_REGISTERED'))
    os.mkdir(os.path.join('HSI_REGISTERED'))
except:
    print('Error making folder!')

# Run through each of the images
for image in pbar:
    # Get base name of file
    f_base = image.split('/')[2].split('.')[0]
    print(f_base)
    # Load RGB image
    rgb = cv2.imread(image)
    # Load Mask
    segmentations = cv2.imread(os.path.join('./SEGMENTATIONS_RAW', f'{f_base}_labeled.jpg'), cv2.IMREAD_UNCHANGED)
    # Load ximea
    ximea_cube = np.load(os.path.join('./VNIR_RAW', f'{f_base}.npy')).reshape((215, 407, 24))
    # Load imec
    imec_cube = np.load(os.path.join('./SWIR_RAW', f'{f_base}.npy')).reshape((168, 211, 9))
    # Generate composite datacube
    cube = generate_registered_cube(ximea_to_rgb, imec_to_rgb, rgb, ximea_cube, imec_cube)
    # Mask the cube for saving
    cube_masked = cube[bounds[0]:bounds[1],bounds[2]:bounds[3],:]
    print(rgb.shape)
    rgb_masked = rgb[bounds[0]:bounds[1],bounds[2]:bounds[3],:]
    # Save the masked cube
    np.savez_compressed(os.path.join('./HSI_REGISTERED', f'{f_base}.npz'), cube=cube_masked)
    # Save the masked rgb image
    cv2.imwrite(os.path.join('./RGB_REGISTERED', f'{f_base}.png'), rgb_masked)
    # Save the registered mask image
    segmentations_masked = segmentations[bounds[0]:bounds[1],bounds[2]:bounds[3]]
    cv2.imwrite(os.path.join('./SEGMENTATIONS_REGISTERED', f'{f_base}.jpg'), segmentations_masked)

  0%|          | 0/504 [00:00<?, ?it/s]

Error making folder!
olin_collection_05_16_23_campus_24
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v3_39


/tmp/ipykernel_1522416/2242120463.py:50: RuntimeWarning: overflow encountered in cast
  data[:,:,idx] = cv2.warpPerspective(


(2054, 2464, 3)
olin_collection_05_16_23_parcel_b_v2_54
(2054, 2464, 3)
olin_collection_05_16_23_parcel_b_0
(2054, 2464, 3)
olin_collection_05_16_23_campus_27
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v2_50
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v2_54
(2054, 2464, 3)
olin_collection_05_16_23_parcel_b_v2_14
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v2_46
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_1
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v4_51
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v2_62
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_52
(2054, 2464, 3)
olin_collection_05_16_23_parcel_b_44
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v3_57
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v4_57
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_v2_22
(2054, 2464, 3)
olin_collection_05_16_23_parcel_b_v2_28
(2054, 2464, 3)
olin_collection_05_15_23_parcel_b_51
(2054, 2464, 3)
olin_collection_05_16_23_campus_38
(2054, 2464, 3)
oli